Juan Pablo Márquez 2022

Tipus de dades que es poden emmagatzemar en una base de dades mongo db.

https://docs.mongodb.com/manual/reference/bson-types/

Mostra les Bases de Dades

In [107]:
show dbs

Hosteleria  0.004GB
RRHH        0.000GB
admin       0.000GB
config      0.000GB
curs_IA_BD  0.000GB
local       0.000GB
test        0.000GB

Utilitzar les bases de dades d'Hoteleria

In [108]:
use Hosteleria

switched to db Hosteleria

Mostrar les col·leccions de la base de dades d'Hoteleria

In [30]:
show collections

restaurants
restaurants30

## Consultes d'agreació

https://docs.mongodb.com/manual/aggregation/

Mostrar el valor màxime d'score

In [16]:
db.restaurants30.aggregate({$group : { _id : "", max_score : { $max:"$grades.score"} }})

{ "_id" : "", "max_score" : [ 38, 10, 7, 13 ] }

Retorna el nombre de valoracions de cada restaurant

https://docs.mongodb.com/v4.2/reference/operator/aggregation/size/

In [67]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   }
] )


{ "_id" : ObjectId("618109795e0862670e9cb829"), "cuisine" : "Delicatessen", "name" : "Wilken'S Fine Food", "numberOfGrades" : 6 }
{ "_id" : ObjectId("618109795e0862670e9cb82a"), "cuisine" : "American ", "name" : "C & C Catering Service", "numberOfGrades" : 4 }
{ "_id" : ObjectId("618109795e0862670e9cb82b"), "cuisine" : "Ice Cream, Gelato, Yogurt, Ices", "name" : "Carvel Ice Cream", "numberOfGrades" : 3 }
{ "_id" : ObjectId("618109795e0862670e9cb82c"), "cuisine" : "Delicatessen", "name" : "Nordic Delicacies", "numberOfGrades" : 3 }
{ "_id" : ObjectId("618109795e0862670e9cb82d"), "cuisine" : "American ", "name" : "The Movable Feast", "numberOfGrades" : 4 }
{ "_id" : ObjectId("618109795e0862670e9cb82e"), "cuisine" : "American ", "name" : "1 East 66Th Street Kitchen", "numberOfGrades" : 4 }
{ "_id" : ObjectId("618109795e0862670e9cb82f"), "cuisine" : "American ", "name" : "Glorious Food", "numberOfGrades" : 5 }
{ "_id" : ObjectId("618109795e0862670e9cb830"), "cuisine" : "Jewish/Kosher", "na

## Aggregation Pipeline

https://docs.mongodb.com/manual/core/aggregation-pipeline/

## Aggregation Pipeline Stages

https://docs.mongodb.com/manual/reference/operator/aggregation-pipeline/

Nombre màxim de valoracions

In [68]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   },
   {
       $group:{ "_id" :"", "maxGrades" : { "$max":"$numberOfGrades"} }
   }
] )


{ "_id" : "", "maxGrades" : 8 }

Nombre mínim de valoracions

In [69]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   },
   {
       $group:{ "_id" :"", "minGrades" : { "$min":"$numberOfGrades"} }
   }
] )

{ "_id" : "", "minGrades" : 3 }

Noms dels restaurant que han rebut més valoracions

In [70]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   },
   {
       $group:{ "_id" :"", 
           "maxGrades" : { "$max":"$numberOfGrades"},
           "restaurant":{"$push": {"_id":"$_id", "name_restaurant":"$name", "numOfGrades":"$numberOfGrades" }}
       }
   }

   
] )

{ "_id" : "", "maxGrades" : 8, "restaurant" : [ { "_id" : ObjectId("618109795e0862670e9cb829"), "name_restaurant" : "Wilken'S Fine Food", "numOfGrades" : 6 }, { "_id" : ObjectId("618109795e0862670e9cb82a"), "name_restaurant" : "C & C Catering Service", "numOfGrades" : 4 }, { "_id" : ObjectId("618109795e0862670e9cb82b"), "name_restaurant" : "Carvel Ice Cream", "numOfGrades" : 3 }, { "_id" : ObjectId("618109795e0862670e9cb82c"), "name_restaurant" : "Nordic Delicacies", "numOfGrades" : 3 }, { "_id" : ObjectId("618109795e0862670e9cb82d"), "name_restaurant" : "The Movable Feast", "numOfGrades" : 4 }, { "_id" : ObjectId("618109795e0862670e9cb82e"), "name_restaurant" : "1 East 66Th Street Kitchen", "numOfGrades" : 4 }, { "_id" : ObjectId("618109795e0862670e9cb82f"), "name_restaurant" : "Glorious Food", "numOfGrades" : 5 }, { "_id" : ObjectId("618109795e0862670e9cb830"), "name_restaurant" : "Seuda Foods", "numOfGrades" : 5 }, { "_id" : ObjectId("618109795e0862670e9cb831"), "name_restaurant" : 

In [71]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   },
   {
       $group:{ "_id" :"", 
           "maxGrades" : { "$max":"$numberOfGrades"},
           "restaurant":{"$push": {"_id":"$_id", "name_restaurant":"$name", "numOfGrades":"$numberOfGrades" }}
       }
   },
   {
       $project:{
           "fitrats":{
               "$filter":{
                   "input":"$restaurant",
                    "as":"res",
                    "cond":{"$eq": ["$$res.numOfGrades","$maxGrades"]}
                }
            }
        }
    }
] )

{ "_id" : "", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb839"), "name_restaurant" : "Ho Mei Restaurant", "numOfGrades" : 8 } ] }

Noms dels restaurant que han rebut més valoracions segons el tipus de cuina

In [73]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         numberOfGrades: { $cond: { if: { $isArray: "$grades" }, then: { $size: "$grades" }, else: "NA"} }
      }
   },
   {
       $group:{ "_id" :"$cuisine", 
           "maxGrades" : { "$max":"$numberOfGrades"},
           "restaurant":{"$push": {"_id":"$_id", "name_restaurant":"$name", "numOfGrades":"$numberOfGrades" }}
       }
   },
   {
       $project:{
           "fitrats":{
               "$filter":{
                   "input":"$restaurant",
                    "as":"res",
                    "cond":{"$eq": ["$$res.numOfGrades","$maxGrades"]}
                }
            }
        }
    }
] )

{ "_id" : "Caribbean", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb83a"), "name_restaurant" : "Shashemene Int'L Restaura", "numOfGrades" : 4 } ] }
{ "_id" : "Irish", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb844"), "name_restaurant" : "Dj Reynolds Pub And Restaurant", "numOfGrades" : 4 } ] }
{ "_id" : "Ice Cream, Gelato, Yogurt, Ices", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb83c"), "name_restaurant" : "Carvel Ice Cream", "numOfGrades" : 5 } ] }
{ "_id" : "Chinese", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb839"), "name_restaurant" : "Ho Mei Restaurant", "numOfGrades" : 8 } ] }
{ "_id" : "Jewish/Kosher", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb830"), "name_restaurant" : "Seuda Foods", "numOfGrades" : 5 } ] }
{ "_id" : "Chicken", "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb836"), "name_restaurant" : "Harriet'S Kitchen", "numOfGrades" : 6 } ] }
{ "_id" : "Hamburgers", "fitrats" : [ { "_id" : ObjectId("618109795e

Puntuació mitjana de cada restaurant.

In [86]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         avgScore :  { "$avg": "$grades.score" }
      }
   }
] )

{ "_id" : ObjectId("618109795e0862670e9cb829"), "cuisine" : "Delicatessen", "name" : "Wilken'S Fine Food", "avgScore" : 10 }
{ "_id" : ObjectId("618109795e0862670e9cb82a"), "cuisine" : "American ", "name" : "C & C Catering Service", "avgScore" : 3.5 }
{ "_id" : ObjectId("618109795e0862670e9cb82b"), "cuisine" : "Ice Cream, Gelato, Yogurt, Ices", "name" : "Carvel Ice Cream", "avgScore" : 10.666666666666666 }
{ "_id" : ObjectId("618109795e0862670e9cb82c"), "cuisine" : "Delicatessen", "name" : "Nordic Delicacies", "avgScore" : 5.666666666666667 }
{ "_id" : ObjectId("618109795e0862670e9cb82d"), "cuisine" : "American ", "name" : "The Movable Feast", "avgScore" : 9.25 }
{ "_id" : ObjectId("618109795e0862670e9cb82e"), "cuisine" : "American ", "name" : "1 East 66Th Street Kitchen", "avgScore" : 3.25 }
{ "_id" : ObjectId("618109795e0862670e9cb82f"), "cuisine" : "American ", "name" : "Glorious Food", "avgScore" : 12.2 }
{ "_id" : ObjectId("618109795e0862670e9cb830"), "cuisine" : "Jewish/Kosher", 

Quina és la puntuació mitjana mes alta

In [87]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         avgScore :  { "$avg": "$grades.score" }
      }
   },
   {
       $group:{ "_id" :"", 
           "maxAvgScore" : { "$max":"$avgScore"},
       }
   }
] )

{ "_id" : "", "maxScoreCuisine" : 27.6 }

Quina és la puntuació mitjana per cada tipus de cuina

In [90]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         avgScore :  { "$avg": "$grades.score" }
      }
   },
   {
       $group:{ "_id" :"$cuisine", 
           "avgScoreCuisine" : { "$avg":"$avgScore"},
       }
   }
] )

{ "_id" : "Irish", "avgScoreCuisine" : 9.25 }
{ "_id" : "Ice Cream, Gelato, Yogurt, Ices", "avgScoreCuisine" : 9.172222222222222 }
{ "_id" : "Jewish/Kosher", "avgScoreCuisine" : 13.35 }
{ "_id" : "Chicken", "avgScoreCuisine" : 10.666666666666666 }
{ "_id" : "Chinese", "avgScoreCuisine" : 18.3625 }
{ "_id" : "Hamburgers", "avgScoreCuisine" : 10 }
{ "_id" : "Delicatessen", "avgScoreCuisine" : 8.383333333333335 }
{ "_id" : "Bakery", "avgScoreCuisine" : 8.2 }
{ "_id" : "Turkish", "avgScoreCuisine" : 10.5 }
{ "_id" : "Caribbean", "avgScoreCuisine" : 8 }
{ "_id" : "American ", "avgScoreCuisine" : 9.43 }

Nom dels restaurant amb puntuació mitjana mes alta segons el tipus de cuina

In [109]:
db.restaurants30.aggregate([
   {
      $project: {  
         name: 1,
         cuisine: 1,
         avgScore :  { "$avg": "$grades.score" }
      }
   },
   {
       $group:{ "_id" :"$cuisine", 
           "maxAvgScoreCuisine" : { "$max":"$avgScore"},
           "restaurant":{"$push": {"_id":"$_id", 
                           "nameRestaurant":"$name", 
                           "typeCuisine":"$cuisine", 
                           "avgScoreRestaurant":"$avgScore" }}
       }
   },
   {
       $project:{"maxAvgScoreCuisine":1,
     
           "fitrats":{
               "$filter":{
                   "input":"$restaurant",
                    "as":"res",
                    "cond":{"$eq": ["$$res.avgScoreRestaurant","$maxAvgScoreCuisine"]}
                }
            }
        }
    }
] )

{ "_id" : "Jewish/Kosher", "maxAvgScoreCuisine" : 17.75, "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb841"), "nameRestaurant" : "Tov Kosher Kitchen", "typeCuisine" : "Jewish/Kosher", "avgScoreRestaurant" : 17.75 } ] }
{ "_id" : "Chicken", "maxAvgScoreCuisine" : 10.666666666666666, "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb836"), "nameRestaurant" : "Harriet'S Kitchen", "typeCuisine" : "Chicken", "avgScoreRestaurant" : 10.666666666666666 } ] }
{ "_id" : "Chinese", "maxAvgScoreCuisine" : 27.6, "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb83d"), "nameRestaurant" : "May May Kitchen", "typeCuisine" : "Chinese", "avgScoreRestaurant" : 27.6 } ] }
{ "_id" : "Irish", "maxAvgScoreCuisine" : 9.25, "fitrats" : [ { "_id" : ObjectId("618109795e0862670e9cb844"), "nameRestaurant" : "Dj Reynolds Pub And Restaurant", "typeCuisine" : "Irish", "avgScoreRestaurant" : 9.25 } ] }
{ "_id" : "Ice Cream, Gelato, Yogurt, Ices", "maxAvgScoreCuisine" : 10.666666666666666, "fitrats" 

Afegeix un camp que sigui la mitja de les puntuacions

In [153]:
db.restaurants30.find().forEach( function(restaurant) 
        {
            n= 0;
            sumaScore = 0;
            restaurant.grades.forEach(function(grade){
                n = n+1;
                sumaScore = sumaScore + grade.score;
            });
            restaurantAvg = sumaScore/n;
            
            db.restaurants30.update({_id:restaurant._id},{$set: {"scoreAvg": restaurantAvg}})})

In [155]:
db.restaurants30.find().pretty()


{
	"_id" : ObjectId("618109795e0862670e9cb829"),
	"address" : {
		"building" : "7114",
		"coord" : [
			-73.9068506,
			40.6199034
		],
		"street" : "Avenue U",
		"zipcode" : "11234"
	},
	"borough" : "Brooklyn",
	"cuisine" : "Delicatessen",
	"grades" : [
		{
			"date" : ISODate("2014-05-29T00:00:00Z"),
			"grade" : "A",
			"score" : 10
		},
		{
			"date" : ISODate("2014-01-14T00:00:00Z"),
			"grade" : "A",
			"score" : 10
		},
		{
			"date" : ISODate("2013-08-03T00:00:00Z"),
			"grade" : "A",
			"score" : 8
		},
		{
			"date" : ISODate("2012-07-18T00:00:00Z"),
			"grade" : "A",
			"score" : 10
		},
		{
			"date" : ISODate("2012-03-09T00:00:00Z"),
			"grade" : "A",
			"score" : 13
		},
		{
			"date" : ISODate("2011-10-14T00:00:00Z"),
			"grade" : "A",
			"score" : 9
		}
	],
	"name" : "Wilken'S Fine Food",
	"restaurant_id" : "40356483",
	"scoreAvg" : 10
}
{
	"_id" : ObjectId("618109795e0862670e9cb82a"),
	"address" : {
		"building" : "7715",
		"coord" : [
			-73.9973325,
			40.61174889999